In [ ]:
%matplotlib inline
from collections import OrderedDict

import numpy as np
import xarray as xr

from pumahu.syn_model import (SynModel,
                              setup_test, setup_realistic,
                              resample, make_sparse)

from pumahu.visualise import trellis_plot
from pumahu.fd import fd
from pumahu.uks import UnscentedKalmanSmoother
from pumahu import get_data

## Simple synthetic model

In [ ]:
xds1 = SynModel(integration_method='euler').run(setup_test(), gradient=True, ignore_cache=True)
xds2 = SynModel(integration_method='rk4').run(setup_test(), gradient=True, ignore_cache=True)
trellis_plot(xds1, xds2)

### Invert the model

In [ ]:
xds1_fd = fd(xds1.exp, use_drmg=True, level2volume=SynModel().synth_fullness)

In [ ]:
trellis_plot(xds1_fd, xds1)

In [ ]:
Q = OrderedDict(T=1e-3, M=1e-3, X=1e-3, q_in=1e1,
                m_in=1e1, m_out=1e1, h=1e-3, W=1e-3,
                dqi=1e4, dMi=1e4, dMo=1e4, dH=0, 
                dW=0)
Q = np.eye(len(Q))*list(Q.values())
uks = UnscentedKalmanSmoother(data=xds2.exp, Q=Q)
xds1_uks = uks(test=True)

In [ ]:
trellis_plot(xds1_uks)#, xds1_inv)

## A more complex synthetic model

In [ ]:
xds3 = SynModel(integration_method='rk4').run(setup_realistic(sinterval=24*60), ignore_cache=True)
trellis_plot(xds3)

In [ ]:
uks = UnscentedKalmanSmoother(data=xds3.exp)
xds3_uks = uks(test=True, smooth=True)

In [ ]:
trellis_plot(xds3_uks, xds3, filled_error=True)

In [ ]:
xds3_fd = fd(xds3.exp, use_drmg=True, level2volume=SynModel().synth_fullness)

In [ ]:
trellis_plot(xds3_fd, xds3, filled_error=True)

### Now we add some noise

In [ ]:
xds4 = SynModel().run(setup_realistic(sinterval=120), addnoise=True, ignore_cache=True)

In [ ]:
trellis_plot(xds4)

### Resample the data as preparation for the inversion

In [ ]:
na = resample(xds4)
na = make_sparse(na, ['m_out', 'm_in', 'X'])
trellis_plot(xds4, xr.Dataset({'exp': na}))

### Invert the resampled data

In [ ]:
Q = OrderedDict(T=1e-2, M=1e1, X=1e-3, q_in=5e2,
                m_in=1e1, m_out=1e1, h=1e-3, W=1e-3,
                dqi=5e3, dMi=1e0, dMo=5e1, dH=0.1, 
                dW=0)
Q = OrderedDict(T=1e-1, M=1e1, X=1e-3, q_in=1e2,
                m_in=1e1, m_out=1e1, h=1e-3, W=1e-3,
                dqi=0, dMi=0, dMo=0, dH=0, 
                dW=0)
Q = np.eye(len(Q))*list(Q.values())

uks4 = UnscentedKalmanSmoother(data=na, Q=Q)
xds4_uks = uks4(test=True, smooth=True, alpha=0.35)

In [ ]:
trellis_plot(xds4_uks, data2=xr.Dataset({'exp': na}), data2_params=['q_in', 'm_out'], filled_error=True)

In [ ]:
xds4_fd = fd(na, use_drmg=True, level2volume=SynModel().synth_fullness)

In [ ]:
trellis_plot(xds4_fd, data2=xr.Dataset({'exp': na}), data2_params=['q_in', 'm_out'], filled_error=True)